In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests, urllib, urllib3, time, warnings
from pvlib.solarposition import get_solarposition
from pvlib import irradiance
from matplotlib.collections import PolyCollection
import knmy
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

print('\014')
plt.close('all')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [2]:
# Convenience dict
number_to_month = { 
    1 : ['January', 31],
    2 : ['February', 28],
    3 : ['March', 31],
    4 : ['April', 30],
    5 : ['May', 31],
    6 : ['June', 30],
    7 : ['July', 31],
    8 : ['August', 31],
    9 : ['September', 30],
   10 : ['October', 31],
   11 : ['November', 30],
   12 : ['December', 31]
}

In [3]:
class Color:
    W       = '\033[0m'
    R       = '\033[1;31;48m'
    G       = '\033[32m'
    B       = '\033[34m'
    O       = '\033[33m'
    LB      = '\033[1;34;48m'
    P       = '\033[35m'
    C       = '\033[36m'
    WBLACK  = '\033[1;37;40m'
    WBLUE   = '\033[1;37;44m'
    END     = '\033[1;37;0m'
    WR      = '\033[1;37;41m'
    BG      = '\033[1;40;42m'

In [4]:
# Load KNMI data, combine YYYYMMDD and HH
start_date_UTC  = 2024010100
end_date_UTC    = 2024123123
visualisation_year = [2024]
visualisation_month = [12]
number_of_weather_station = 330
print(f'start date UTC = {str(start_date_UTC)}')
print(f'end date UTC = {str(end_date_UTC)}')

start date UTC = 2024010100
end date UTC = 2024123123


In [5]:
url = 'https://www.knmi.nl/nederland-nu/klimatologie/uurgegevens'
# Dutch: Uurvak u loopt van u - 1 UT tot u UT.
try:
    urllib.request.urlretrieve(url)
    disclaimer, stations, variables, df = knmy.knmy.get_hourly_data(
        stations = [number_of_weather_station], start = start_date_UTC, end = end_date_UTC,
        inseason = False, variables = ['WIND', 'TEMP', 'Q', 'SQ', 'VICL', 'P'], parse = True)
    # When KNMI is updating its data, the above statement will produce an empty dataframe.
    # In this case it's preferable to load the dataframe from the former successfull run.
    if df['YYYYMMDD'].isnull().values.any():
        print(Color.R + 'loading KNMI data from file ...' + Color.END)
        df = pd.read_csv('~/Documents/Soleil/Lectoraar-Internship/Solar-Panel-Project/Energy/year_data/df_KNMI.csv')
    else:
        station_name = stations['name'].values[0]
        print(f'Loading data from weather station: {station_name}')
        print('Saving KNMI data to file ...')
        df.to_csv('~/Documents/Soleil/Lectoraar-Internship/Solar-Panel-Project/Energy/data_experimenting/year_data/df_KNMI.csv')
except urllib.error.HTTPError as err:
    print(f'{url}: {err}')
    print(Color.R + 'loading KNMI data from file ...' + Color.END)
    df = pd.read_csv('~/Documents/Soleil/Lectoraar-Internship/Solar-Panel-Project/Energy/data_experimenting/year_data/df_KNMI.csv')

Loading data from weather station: Hoek van Holland
Saving KNMI data to file ...


In [6]:
# Date formatting
df['HH'] = df['HH'].astype(str).str.zfill(2)
df['YYYYMMDDHH'] = df['YYYYMMDD'].astype(str) + df['HH'].astype(str).replace('24','00')
df['YYYYMMDDHH'] = pd.to_datetime(df['YYYYMMDDHH'], format = '%Y%m%d%H', errors = 'coerce')
df['YYYYMMDDHH'] = np.where(df['YYYYMMDDHH'].dt.strftime('%H:%M') == '00:00',
                       df['YYYYMMDDHH'] + pd.DateOffset(days = 1),
                       df['YYYYMMDDHH'])

In [7]:
drop_columns = ['YYYYMMDD', 'HH', 'DD', 'T10N', 'TD', 'VV', 'STN']
df = df.drop(columns = drop_columns)
df.insert(0, 'YYYYMMDDHH', df.pop('YYYYMMDDHH'))
print(df.head())

           YYYYMMDDHH     FH     FF     FX   T  Q  SQ   N   U       P
0 2024-01-01 01:00:00  120.0  120.0  180.0  75  0   0 NaN  91  9951.0
1 2024-01-01 02:00:00   90.0  100.0  150.0  74  0   0 NaN  91  9958.0
2 2024-01-01 03:00:00  100.0  110.0  160.0  92  0   0 NaN  79  9959.0
3 2024-01-01 04:00:00  110.0  110.0  190.0  91  0   0 NaN  80  9966.0
4 2024-01-01 05:00:00  100.0  100.0  150.0  95  0   0 NaN  76  9973.0


In [8]:
# New column names 
new_column_names = { 
    'FH' : 'average windspeed (m/s)',
    'FF' : 'windspeed previous 10 minuten (m/s)',
    'FX' : 'highest windspeed (m/s)',
    'T'  : 'temperature in degrees Celcius',
    'SQ' : 'duration of sunshine (h)',
    'Q'  : 'global radiation (in J/m^2)',    
    'P'  : 'air pressure (Pa)',
    'N'  : 'cloud cover (-)',
    'U'  : 'relative humidity (in percentages)'
}
df = df.rename(columns = new_column_names)

In [9]:
df.sample(5)

,YYYYMMDDHH,average windspeed (m/s),windspeed previous 10 minuten (m/s),highest windspeed (m/s),temperature in degrees Celcius,global radiation (in J/m^2),duration of sunshine (h),cloud cover (-),relative humidity (in percentages),air pressure (Pa)
5446,2024-08-14 23:00:00,50.0,40.0,60.0,187,0,0,NaN,89,10155.0
8634,2024-12-25 19:00:00,40.0,40.0,70.0,89,0,0,NaN,97,10352.0
2013,2024-03-24 22:00:00,90.0,90.0,110.0,74,0,0,NaN,73,10084.0
1960,2024-03-22 17:00:00,30.0,30.0,40.0,88,14,0,NaN,83,10145.0
928,2024-02-08 17:00:00,80.0,70.0,130.0,34,1,0,NaN,96,9901.0


In [10]:
# Group variables
windvariabelen = ['average windspeed (m/s)',
                  'windspeed previous 10 minuten (m/s)', 
                  'highest windspeed (m/s)' ]
temperature_variables = ['temperature in degrees Celcius'],
air_pressurevariabelen = ['air pressure (Pa)']
zonvariabelen = ['duration of sunshine (h)', 'global radiation (in J/m^2)']

# Physical constants
area_of_solar_panel = 2.45         ## area of one module [m^2]
efficiency_solar_panel = 0.20  ## efficiency of one solar panel [-]
number_of_solar_panels = 2   ## total number of solar panels used [-]
lat = 51.96811434560797             ## latitude of plant
lon = 4.095795682209092             ## longitude of plant
module = \
    {'theta_M': np.deg2rad(20),
     'azimuth': np.deg2rad(115)}

# Convert to S.I. units
for column in zonvariabelen:
    if column == 'duration of sunshine (h)':
        df[column] *= 0.10
    elif column == 'global radiation (in J/m^2)':
        df[column] *= 1e4

In [11]:
# Get the elevation of the module
def make_remote_request(url: str, params: dict):
    global response
    while True:
        try:
            response = requests.get(url + urllib.parse.urlencode(params))
        except (OSError, urllib3.exceptions.ProtocolError) as error:
            print(error)
            continue
        break

    return response

def elevation_function(lat, lon):
    url = 'https://api.opentopodata.org/v1/eudem25m?'
    params = {'locations': f"{lat},{lon}"}
    result = make_remote_request(url, params)
    if 'results' in result.json().keys():
        return_value = result.json()['results'][0]['elevation']
    else:
        return_value = None
    return return_value

In [12]:
elevation_module = elevation_function(lat, lon)
while elevation_module == None:
    time.sleep(0.40)
    elevation_module = elevation_function(lat, lon)

In [13]:
# Calculate the positions of the sun
date_time_or_doy = df['YYYYMMDDHH'] 
solpos = get_solarposition(
    time = date_time_or_doy, latitude = lat,
    longitude = lon, altitude = elevation_module,
    pressure = df['air pressure (Pa)'].values, 
    temperature = df['temperature in degrees Celcius'].values) 

In [14]:
solpos.head()

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
YYYYMMDDHH,,,,,,
2024-01-01 01:00:00,147.847274,147.847274,-57.847274,-57.847274,32.920517,-3.095135
2024-01-01 02:00:00,141.460269,141.460269,-51.460269,-51.460269,54.207034,-3.114902
2024-01-01 03:00:00,133.271369,133.271369,-43.271369,-43.271369,70.685595,-3.134661
2024-01-01 04:00:00,124.266964,124.266964,-34.266964,-34.266964,84.137301,-3.154412
2024-01-01 05:00:00,115.042669,115.042669,-25.042669,-25.042669,95.966753,-3.174155


#### Adding 2 columns for calculated `zenith` and `azimuth`

In [15]:
df['solpos_zenith'] = solpos['zenith'].values
df['solpos_azimuth'] = solpos['azimuth'].values

In [16]:
# Orientation parameters
theta_M = module['theta_M']                     ## tilt angle module
az_M = module['azimuth']                        ## azimuth angle module

# Compute G_module, first decompose the ghi into dni and dhi
ghi = df['global radiation (in J/m^2)'].values  ## global horizontal irradiance in [J/m^2]
out_erbs = irradiance.erbs(ghi, solpos.zenith, solpos.index)
out_erbs = out_erbs.rename(columns = {'dni': 'dni_erbs', 'dhi': 'dhi_erbs'})
dni = out_erbs['dni_erbs'].values               ## direct normal irradiance. [J/m^2]
dhi = out_erbs['dhi_erbs'].values               ## diffuse horizontal irradiance in [J/m^2]
sky_model = 'isotropic'
poa = irradiance.get_total_irradiance(np.rad2deg(theta_M), np.rad2deg(az_M),
                         solpos.zenith, solpos.azimuth,
                         dni, ghi, dhi, dni_extra = None, airmass = None,
                         surface_type = 'sea',
                         model = sky_model)
G_module = poa['poa_global'].values

In [17]:
# Store into the dataframe
df['yield solar panel (kWh)'] = \
    (G_module * area_of_solar_panel * number_of_solar_panels * efficiency_solar_panel) / (1e3 * 3600 )
print("Last data frame after calculations/n")

Last data frame after calculations/n


In [18]:
df.sample(5)

,YYYYMMDDHH,average windspeed (m/s),windspeed previous 10 minuten (m/s),highest windspeed (m/s),temperature in degrees Celcius,global radiation (in J/m^2),duration of sunshine (h),cloud cover (-),relative humidity (in percentages),air pressure (Pa),solpos_zenith,solpos_azimuth,yield solar panel (kWh)
7406,2024-11-04 15:00:00,40.0,40.0,60.0,128,600000.0,1.0,NaN,74,10259.0,81.784831,231.177953,0.026433
1948,2024-03-22 05:00:00,90.0,90.0,130.0,96,0.0,0.0,NaN,92,10183.0,97.075658,79.509715,0.000000
8065,2024-12-02 02:00:00,80.0,80.0,110.0,98,0.0,0.0,NaN,94,10129.0,138.840013,57.424080,0.000000
4127,2024-06-21 00:00:00,50.0,40.0,60.0,144,0.0,0.0,NaN,94,10165.0,104.528447,3.451951,0.000000
5510,2024-08-17 15:00:00,30.0,40.0,40.0,226,2180000.0,1.0,NaN,53,10129.0,54.588345,242.817231,0.415521


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   YYYYMMDDHH                           8784 non-null   datetime64[ns]
 1   average windspeed (m/s)              8754 non-null   float64       
 2   windspeed previous 10 minuten (m/s)  8754 non-null   float64       
 3   highest windspeed (m/s)              8754 non-null   float64       
 4   temperature in degrees Celcius       8784 non-null   int64         
 5   global radiation (in J/m^2)          8784 non-null   float64       
 6   duration of sunshine (h)             8784 non-null   float64       
 7   cloud cover (-)                      0 non-null      float64       
 8   relative humidity (in percentages)   8784 non-null   int64         
 9   air pressure (Pa)                    8678 non-null   float64       
 10  solpos_zenit

#### Droping `cloud cover (-)` and `air pressure (Pa)` columns since they contains only null values

In [22]:
df=df.drop(["cloud cover (-)", "air pressure (Pa)"],axis = "columns")

In [23]:
df.loc[df.isnull().any(axis=1)]

,YYYYMMDDHH,average windspeed (m/s),windspeed previous 10 minuten (m/s),highest windspeed (m/s),temperature in degrees Celcius,global radiation (in J/m^2),duration of sunshine (h),relative humidity (in percentages),solpos_zenith,solpos_azimuth,yield solar panel (kWh)
2965,2024-05-03 14:00:00,NaN,NaN,NaN,110,460000.0,0.0,90,45.407517,230.577882,0.102859
2966,2024-05-03 15:00:00,NaN,NaN,NaN,117,610000.0,0.0,84,53.279705,246.569251,0.114983
2967,2024-05-03 16:00:00,NaN,NaN,NaN,114,620000.0,0.0,83,62.110926,260.096298,0.085050
2968,2024-05-03 17:00:00,NaN,NaN,NaN,119,550000.0,0.2,78,71.306785,272.213051,0.025198
2969,2024-05-03 18:00:00,NaN,NaN,NaN,116,290000.0,0.0,77,80.434360,283.739340,0.012776
2970,2024-05-03 19:00:00,NaN,NaN,NaN,114,90000.0,0.2,78,89.122169,295.305559,0.023806
2971,2024-05-03 20:00:00,NaN,NaN,NaN,113,0.0,0.0,77,96.994763,307.410432,0.000000
2972,2024-05-03 21:00:00,NaN,NaN,NaN,107,0.0,0.0,84,103.636870,320.424490,0.000000
2973,2024-05-03 22:00:00,NaN,NaN,NaN,101,0.0,0.0,87,108.594621,334.522350,0.000000
2974,2024-05-03 23:00:00,NaN,NaN,NaN,100,0.0,0.0,82,111.433245,349.569992,0.000000
